In [1]:
# load library
import requests
import os
from datetime import datetime
from dateutil.relativedelta import relativedelta, FR
import fitz
from bs4 import BeautifulSoup
from PIL import Image
from io import BytesIO
import time

# download file from url
def pdfDownload(save_path, url, headers, file_nm, param=None, retries=3):
	resp = None

	try:
		resp = requests.get(url, params=param, headers=headers)
		resp.raise_for_status()

		pdf_layer = save_path +  file_nm + ".pdf"

		with open(pdf_layer, 'wb') as f:
			f.write(resp.content)
		#print("complete writing pdf!")

	except requests.exceptions.HTTPError as e:
		if 500 <= resp.status_code < 600 and retries > 0:
			print('Retries : {0}'.format(retries))
			return getDownload(url, param, retries - 1)
		else:
			return resp.status_code
	return resp

def importImgFromPDF(save_path, file, start_page = 1, end_page = 1):

	doc = fitz.open(file)

	# get img page by
	for i in range( start_page-1, end_page ):
		for img in doc.getPageImageList(i):
			xref = img[0]
			pix = fitz.Pixmap(doc, xref)
			if pix.n < 5:  # this is GRAY or RGB
				pix.writePNG(save_path + "p%s-%s.png" % (i, xref))
			else:  # CMYK: convert to RGB first
				pix1 = fitz.Pixmap(fitz.csRGB, pix)
				pix1.writePNG(save_path + "p%s-%s.png" % (i, xref))
				pix1 = None
			pix = None

	#print("img from PDF has been saved!")

def importImgFromURL(save_path, url, file_nm):

	response = requests.get(url)
	img = Image.open(BytesIO(response.content))
	img_save_path = save_path + file_nm + '.png'
	img.save(img_save_path)
	#print("img from URL has been saved!")

# download file from url
def fredREQ(url, headers, param=None, retries=3):
	resp = None

	try:
		resp = requests.get(url, params=param, headers=headers)
		resp.raise_for_status()
	except requests.exceptions.HTTPError as e:
		if 500 <= resp.status_code < 600 and retries > 0:
			print('Retries : {0}'.format(retries))
			return fredREQ(url, param, retries - 1)
		else:
			return resp.status_code
	return resp


In [4]:

# 3. Fear & Greed
url = "https://money.cnn.com/data/fear-and-greed/"
headers = {'Referer': url,
           'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36'}

req = fredREQ(url, headers)
req_txt = req.text
soup = BeautifulSoup(req_txt, 'html.parser')

id_url = soup.find_all("div", {"id": "needleChart"})

style_url = id_url[0]['style']
img_url = style_url.split("('", 1)[1].split("')")[0]

fear_greed_nm = "FG_image" #+ today_str
importImgFromURL("./", img_url, fear_greed_nm)

UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x7f6ab5b2f830>

In [5]:
img_url

'http://markets.money.cnn.com/Marketsdata/uploadhandler/z6f8f7c0azc3454a6b12ab4a64b2a4b65d438fec2d.png'